In [1]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.helper_funcs import candles_to_df
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.ex_strat_03 import SMACrossing
from datetime import datetime


%load_ext autoreload
%autoreload 2

In [2]:
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

In [3]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="30m",
    candles_to_dl=3000,
    # since_datetime=datetime(2023, 11, 14, 19),
    # until_datetime=datetime(2023, 11, 16),
)

candles_to_df(candles=candles)

,timestamp,open,high,low,close,volume
datetime,,,,,,
2023-10-13 03:30:00,1697167800000,26786.4,26788.6,26756.6,26769.4,12.460
2023-10-13 04:00:00,1697169600000,26769.4,26795.1,26765.3,26784.3,7.698
2023-10-13 04:30:00,1697171400000,26784.3,26796.0,26776.2,26791.9,5.769
2023-10-13 05:00:00,1697173200000,26791.9,26811.4,26784.6,26788.8,8.060
2023-10-13 05:30:00,1697175000000,26788.8,26815.1,26770.1,26809.5,7.658
...,...,...,...,...,...,...
2023-12-14 13:00:00,1702558800000,43100.1,43106.8,41928.2,42465.2,1299.741
2023-12-14 13:30:00,1702560600000,42465.2,42634.0,41298.1,42569.1,2938.656
2023-12-14 14:00:00,1702562400000,42569.1,42695.2,42352.7,42620.7,768.874


In [23]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([0]),
    sl_based_on_add_pct=np.array([0, 0.05, 0.1, 0.15]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="market",
    tp_strategy_type=TakeProfitStrategyType.Provided,
    trail_sl_bool=False,
    z_or_e_type=None,
)
strategy = SMACrossing(
    long_short="long",
    sma_fast_length=np.arange(20, 61, 20),
    sma_slow_length=np.arange(30, 91, 30),
)

In [24]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    logger_bool=False,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 7
Total order settings to test: 8
Total combinations of settings to test: 56
Total candles: 3,000
Total candles to test: 168,000


In [25]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
2,0,2,71.0,28,43,93.378,39.437,0.821,461.946,933.775,1933.775
0,0,0,71.0,26,45,45.578,36.620,0.808,389.186,455.776,1455.776
6,0,6,71.0,29,42,111.291,40.845,0.797,380.807,1112.911,2112.911
4,0,4,71.0,29,42,124.780,40.845,0.776,418.151,1247.803,2247.803
7,0,7,70.0,29,41,74.543,41.429,0.724,240.479,745.426,1745.426
5,0,5,70.0,29,41,77.234,41.429,0.707,253.196,772.337,1772.337
3,0,3,70.0,29,41,81.171,41.429,0.690,268.342,811.708,1811.708
1,0,1,70.0,29,41,87.029,41.429,0.672,287.757,870.293,1870.293
15,1,7,27.0,16,11,83.662,59.259,0.666,76.754,836.621,1836.621
9,1,1,27.0,16,11,95.988,59.259,0.618,89.731,959.879,1959.879


In [26]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=0,
    ind_set_index=0,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 5,
    risk_account_pct_size = 0.03,
    risk_reward = 0.0,
    sl_based_on_add_pct = 0.0,
    sl_based_on_lookback = 20,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.005,
    trail_sl_when_pct = 0.01,
)


In [8]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,1,7,108,1697362200000,2023-10-15 09:30:00,EntryFilled,1000.000,948.043,4318.091,51.957,...,4370.048,26886.3,NaN,0.163,4370.048,NaN,0.600,26733.9,NaN,NaN
1,1,7,133,1697407200000,2023-10-15 22:00:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.005,27028.6,NaN,NaN
2,1,7,135,1697410800000,2023-10-15 23:00:00,StopLossFilled,1017.923,1017.923,NaN,NaN,...,NaN,NaN,27028.6,NaN,NaN,17.923,NaN,NaN,NaN,NaN
3,1,7,307,1697720400000,2023-10-19 13:00:00,EntryFilled,1017.923,978.649,1844.276,39.274,...,1883.550,28460.5,NaN,0.066,1883.550,NaN,1.500,28041.1,NaN,NaN
4,1,7,334,1697769000000,2023-10-20 02:30:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.005,28609.6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,1,7,2961,1702497600000,2023-12-13 20:00:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.007,42367.1,NaN,NaN
139,1,7,2962,1702499400000,2023-12-13 20:30:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.008,42417.8,NaN,NaN
140,1,7,2963,1702501200000,2023-12-13 21:00:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.011,42536.6,NaN,NaN
141,1,7,2965,1702504800000,2023-12-13 22:00:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.017,42768.9,NaN,NaN
